## iou ve conf optimizasyonu

In [1]:
from pathlib import Path
import os, sys
import pandas as pd
from ultralytics import YOLO

sys.path = [p for p in sys.path if p and p != r"C:\HysoExpect"]

best_pt = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\outputs\runs\finetune_y8s_832_lr1e-3_nomosaic\weights\best.pt")
dataset_root = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced")
test_images = dataset_root / "images" / "test"
test_labels = dataset_root / "labels" / "test"

base_out = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection")

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

def count_gt(label_path: Path) -> int:
    if not label_path.exists():
        return 0
    lines = [ln.strip() for ln in label_path.read_text(encoding="utf-8", errors="ignore").splitlines()]
    return sum(1 for ln in lines if ln)

def run_summary(conf: float, iou: float, imgsz: int = 832, device: str = "cuda:0", limit: int | None = None):
    tag = f"conf{int(round(conf*100)):02d}_iou{int(round(iou*100)):02d}"
    out_root = base_out / f"sum_{tag}"
    rep_out = out_root / "reports"
    rep_out.mkdir(parents=True, exist_ok=True)

    model = YOLO(str(best_pt))

    imgs = [p for p in test_images.iterdir() if p.suffix.lower() in IMG_EXTS]
    imgs = sorted(imgs)
    if limit is not None:
        imgs = imgs[: int(limit)]

    gt_total = 0
    pred_total = 0
    abs_err_sum = 0.0
    sq_err_sum = 0.0
    mape_sum = 0.0
    mape_n = 0

    for img_path in imgs:
        stem = img_path.stem
        gt = count_gt(test_labels / f"{stem}.txt")
        gt_total += gt

        r = model.predict(
            source=str(img_path),
            conf=conf,
            iou=iou,
            imgsz=imgsz,
            device=device,
            save=False,
            verbose=False,
        )[0]
        pred = int(r.boxes.shape[0]) if r.boxes is not None else 0
        pred_total += pred

        err = pred - gt
        ae = abs(err)
        abs_err_sum += ae
        sq_err_sum += (err * err)

        if gt > 0:
            mape_sum += (ae / gt)
            mape_n += 1

    n = len(imgs)
    mae = abs_err_sum / n if n else 0.0
    rmse = (sq_err_sum / n) ** 0.5 if n else 0.0
    mape = (mape_sum / mape_n) if mape_n else 0.0

    summary = pd.DataFrame(
        [
            {"metric": "MAE", "value": mae},
            {"metric": "RMSE", "value": rmse},
            {"metric": "MAPE(ignore gt=0)", "value": mape},
            {"metric": "N_images", "value": float(n)},
            {"metric": "GT_total", "value": float(gt_total)},
            {"metric": "Pred_total", "value": float(pred_total)},
        ]
    )

    summary.to_csv(rep_out / "count_summary.csv", index=False, encoding="utf-8")
    print(f"[OK] {tag} -> {rep_out / 'count_summary.csv'}")
    return summary

runs = [
    (0.35, 0.55),
    (0.35, 0.50),
    (0.35, 0.45),
    (0.40, 0.50),
]

all_summaries = []
for conf, iou in runs:
    s = run_summary(conf=conf, iou=iou)
    s2 = s.copy()
    s2["conf"] = conf
    s2["iou"] = iou
    all_summaries.append(s2)

pd.concat(all_summaries, ignore_index=True)

[OK] conf35_iou55 -> C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\sum_conf35_iou55\reports\count_summary.csv
[OK] conf35_iou50 -> C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\sum_conf35_iou50\reports\count_summary.csv
[OK] conf35_iou45 -> C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\sum_conf35_iou45\reports\count_summary.csv
[OK] conf40_iou50 -> C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\sum_conf40_iou50\reports\count_summary.csv


,metric,value,conf,iou
0,MAE,1.664286,0.35,0.55
1,RMSE,4.566806,0.35,0.55
2,MAPE(ignore gt=0),0.072717,0.35,0.55
3,N_images,700.000000,0.35,0.55
4,GT_total,18605.000000,0.35,0.55
5,Pred_total,18864.000000,0.35,0.55
6,MAE,1.667143,0.35,0.50
7,RMSE,4.522799,0.35,0.50
8,MAPE(ignore gt=0),0.072933,0.35,0.50
9,N_images,700.000000,0.35,0.50
